# Red Wine Quality Classifier
## Introduction:
Red wine is a popular type of wine made from dark-colored grapes, known for its rich flavor and complexity. Wine quality is typically evaluated based on factors such as aroma, taste, and acidity,etc. We use this dataset to address “how can a business predict the quality of wine they produce based on its chemical composition by using this dataset”.


There are 12 input variables in this dataset based on physicochemical tests: 

        1 - fixed acidity ((grams of tartaric acid)/dm3): refers to the non-volatile acids present in wine, which do not evaporate easily.
        2 - volatile acidity ((grams of acetic acid)/dm3): refers to the amount of acetic acid in wine, which can lead to an unpleasant vinegar taste if present in excessive amounts.       
        3 - citric acid (g/dm3): is found in small quantities in wine and can add a sense of freshness and flavor to it.
        4 - residual sugar (g/dm3): refers to the amount of sugar left over after the fermentation process is complete,wines with more than 45 grams per liter are considered sweet. 
        5 - chlorides(In grams of sodium chloride per cubic decimetre): refer to the amount of salt present in wine.       
        6 - free sulfur dioxide (mg/dm3): helps prevent microbial growth and wine oxidation. 
        7 - total sulfur dioxide (mg/dm3): refers to the amount of both free and bound forms of SO2. Concentrations over 50 ppm can be detected in the taste and aroma of wine.
        8 - density (g/cm3): Density of wine is close to that of water and depends on its alcohol and sugar content.
        9 - pH :measures the acidity or basicity of wine on a scale from 0 to 14.
        10 - sulphates ((grams of potassium sulphate)/dm3): are an additive in wine that can contribute to sulfur dioxide gas levels, acting as an antimicrobial and antioxidant. 
        11 - alcohol: percentage of alcohol content is an important factor to consider when assessing wine quality. 
        Output variable (based on sensory rating): 
        12 - quality (between 0-10): Sensory rating by human experts.
 
## Methods:
First split the dataset into training and dataset. Use training set to preprocess and create model specification. Next use cross-validation on training set to determine the best K nearest-neighbor value to use when creating a classifier, which ensures the highest prediction accuracy. Lastly, use the tuned classifier with K value that gives the highest prediction accuracy on the testing set to predict the quality of the newly introduced wine.

The columns we will use when classifying will be pH, alcohol, sulphates, residual sugar, volatile acidity, and quality. We chose to use these columns because these qualities will significantly ater the human assessed quality of a new wine.  


Visualize the result with a line plot that includes the different wines numbered 1 to n, where n is the total number of wines used (ordered, distinct numerical values), on the x-axis and their ratings from 0-10 on the y-axis (distinct numerical values). 
## Expected outcomes and significance:
In this research, we expected to find which physicochemical properties make high-quality red wine, and what chemical compositions are required for high-quality red wine. Once we know what physicochemical properties contribute to a good quality score, businesses will be able to use this information to help them produce higher-quality wine. Based on our findings, we came up with two questions that can be worked on in the future:
“What role does each chemical composition play in making high-quality red wine?”
“What relationships between these properties contribute to making high-quality red wine?”

### Citation
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

## Preliminary exploratory data analysis: 

In [ ]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

wine_quality <- read_delim("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", delim = ";")
colnames(wine_quality) <- make.names(colnames(wine_quality))
wine_quality

wine_quality <- select(wine_quality, volatile.acidity, pH, alcohol, sulphates, residual.sugar, quality) |>
                         mutate(across(1:6, as.numeric)) |>
                        mutate(new_quality = case_when(
                                  quality <= 5 ~ "bad",
                      quality > 5 & quality < 7  ~ "moderate",
                                quality >= 7 ~ "good",
                                  TRUE ~ "")) |>
                        arrange(quality)
wine_quality

### Explanations 

Summary table including mean column values, grouped by quality.
Plot of number of observations for each quality of wine.

Scatter plots of the distribution for each predictor, showing where the data lies for each wine quality. This will help determine what values for each physiochemical property contribute to a better quality wine.

In [ ]:
library(RColorBrewer)

wine_quality_factored <- mutate(wine_quality, quality = as.factor(quality))
wine_quality_factored

wine_summary <- group_by(wine_quality, new_quality) |>
                summarize(mean_pH = mean(pH), 
                          mean_acidity = mean(volatile.acidity), 
                          mean_alcohol = mean(alcohol),
                         mean_sulphates = mean(sulphates),
                         mean_sugar = mean(residual.sugar))
wine_summary

wine_grouped <- group_by(wine_quality, quality) |>
                summarize(observations = n())
wine_grouped

options(repr.plot.width = 10, repr.plot.height = 8)
quality_num_plot <- ggplot(wine_grouped, aes(x = quality, y = observations)) +
                        geom_bar(stat = "identity") +
                        xlab("Quality Based on Sensory Rating (0-10)") +
                        ylab("Number of Observations") + 
                        theme(text = element_text(size=14))
quality_num_plot

acidity_quality_plot <- ggplot(wine_quality, aes(x = volatile.acidity, y = quality)) +
                        geom_point(alpha = 0.3) +
                        labs(x = "Volatile Acidity (g(acetic acid)/dm3)", 
                         y = "Quality Sensory Rating (1-10)", 
                         title = "Volatile Acidity by Quality") +
                        theme(text = element_text(size=14))
acidity_quality_plot

pH_quality_plot <- ggplot(wine_quality, aes(x = pH, y = quality)) +
                        geom_point(alpha = 0.3) +
                        labs(x = "pH (1-14)", 
                         y = "Quality Sensory Rating (1-10)", 
                         title = "pH by Quality") +
                        theme(text = element_text(size=14))
pH_quality_plot

sugar_quality_plot <- ggplot(wine_quality, aes(x = residual.sugar, y = quality)) +
                        geom_point(alpha = 0.3) +
                        labs(x = "Residual Sugar (g/dm3)", 
                         y = "Quality Sensory Rating (1-10)", 
                         title = "Residual Sugar by Quality") +
                        theme(text = element_text(size=14))
sugar_quality_plot

alc_quality_plot <- ggplot(wine_quality, aes(x = alcohol, y = quality)) +
                        geom_point(alpha = 0.3) +
                        labs(x = "Alcohol Content (%)", 
                         y = "Quality Sensory Rating (1-10)", 
                         title = "Alcohol Percentage by Quality") +
                        theme(text = element_text(size=14))
alc_quality_plot

sulph_quality_plot <- ggplot(wine_quality, aes(x = sulphates, y = quality)) +
                        geom_point(alpha = 0.3) +
                        labs(x = "Sulphate Level (g(potassium sulphate)/dm3)", 
                         y = "Quality Sensory Rating (1-10)", 
                         title = "Sulphate Level by Quality") +
                        theme(text = element_text(size=14))
sulph_quality_plot

In [ ]:
# correlations between variables
library(GGally)
options(repr.plot.height = 20, repr.plot.width = 21)
wine_quality <- wine_quality_factored |>
    select(volatile.acidity:new_quality)
wine_pairs <- ggpairs(wine_quality) +
                ggtitle("Correlations Between Variables") +
                theme(text = element_text(size = 20))
wine_pairs

In [ ]:
wine_quality_new <-wine_quality|>
    select(-quality)|>
    mutate(new_quality = as.factor(new_quality))
wine_quality_new


data_split <- initial_split(wine_quality_new, prop = 0.75, strata = new_quality) 
data_train <- training(data_split)
data_test <- testing(data_split)

In [ ]:
my_recipe <- recipe(new_quality ~ . , data = data_train) |> 
    step_scale(all_predictors()) |> 
    step_center(all_predictors())

# specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
    set_engine("kknn") |>
    set_mode("classification")

# vfold
my_vfold <- vfold_cv(data_train, v = 5, strata = new_quality)

# k_values
k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))



# k validations
knn_fit <- workflow() |>
    add_recipe(my_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = my_vfold, grid = k_vals) |>
    collect_metrics()

knn_fit

In [ ]:
df_knn_fit <- knn_fit |>
    arrange(desc(mean))
    df_knn_fit

In [ ]:
knn_spec2 <- nearest_neighbor(weight_func = "rectangular", neighbors = 6) |>
set_engine("kknn") |>
set_mode("classification")
#creating second specification with neighbors value decided from above

knn_fit2 <- workflow() |>
add_recipe(my_recipe) |>
add_model(knn_spec2) |>
fit(data = data_train)
knn_fit2

In [ ]:
#testing classifier on test data set
#testing the accuracy of the model on the test data set
data_test_predictions <- predict(knn_fit2, data_test) |>
    bind_cols(data_test) 
data_test_predictions

In [ ]:

glimpse(data_test_predictions)

In [ ]:
write_csv(data_test_predictions, "test_predictions.csv")

In [ ]:
# Compare the accuracy of predictions to the true values in the test set
accuracies <- data_test_predictions |> 
    metrics(truth = new_quality, estimate = .pred_class) |> 
    select(.metric, .estimate) |> 
    head(1)
accuracies

# Compare the predictions to the true values in a confusion matrix
wine_cm <- data_test_predictions |> 
    conf_mat(truth = new_quality, estimate = .pred_class)
wine_cm